# Beamsize minimization with Xopt

## define the diagnostic

In [ ]:
from plugins.interfaces.diagnostics import EPICSImageDiagnostic, ROI

screen_name = "DYG15"
save_image_location = ""
roi = ROI(
    xmin=0, xmax=100, ymin=0, ymax=100
)
charge_pvs = []

image_diagnostic = EPICSImageDiagnostic(
    screen_name=screen_name,
    resolution_suffix=None,
    roi=roi,
    extra_pvs=charge_pvs,
    save_image_location=save_image_location
)

In [ ]:
image_diagnostic.test_measurement()

### Define the evaluator

In [1]:
from epics import caput, caget
from xopt import Evaluator
import time
import numpy as np

IMAGE_CONSTRAINTS = {
            "bb_penalty": ["LESS_THAN", 0.0],
            "log10_total_intensity": ["GREATER_THAN", image_diagnostic.min_log_intensity]
        }

def evaluate_function(inputs: dict) -> dict:
    global image_diagnostic
    # caput values
    for name, val in inputs:
        caput(name, val)

    # wait for changes to occur
    time.sleep(2)

    results = image_diagnostic.measure_beamsize(5, **inputs)

    # get other PV's NOTE: Measurements not synchronous with beamsize measurements!
    results = results

    # add total beam size
    results["total_size"] = np.sqrt(np.array(results["Sx"]) ** 2 + np.array(results["Sy"]) ** 2)
    return results

evaluator = Evaluator(function=evaluate_function)

### Define VOCS
Here we define the names and ranges of input parameters, the names and settings of
objectives, and the names and settings of constraints. Note that the keys here should
 be referenced in the evaluate function above.

In [2]:
from xopt import VOCS

# define control PVs and ranges here
vocs = VOCS(
    variables = {
        "x1": [0, 1],
        "x2": [0, 1],
        "x3": [0, 1]
    },
    objectives = {"total_size":"MINIMIZE"},
    constraints = IMAGE_CONSTRAINTS
)

### Define the Generator

In [3]:
from xopt.generators import get_generator
from xopt.generators.bayesian.models.standard import StandardModelConstructor

model_constructor = StandardModelConstructor(use_low_noise_prior=False)
generator = get_generator("expected_improvement")(vocs=vocs, gp_constructor=model_constructor)


    Random number generator.
    


###  Combine into Xopt object

In [10]:
from xopt import Xopt
dump_filename = "my_data.yml"
X = Xopt(vocs=vocs, generator=generator, evaluator=evaluator, dump_file=dump_filename)

## Introspection
Objects in Xopt can be printed to a string or dumped to a text file for easy
introspection of attributes and current configuration.

In [11]:
# Convenient representation of the state.
X


            Xopt
________________________________
Version: 2.1.0+5.g375e990d
Data size: 0
Config as YAML:
dump_file: my_data.yml
evaluator:
  function: __main__.evaluate_function
  function_kwargs: {}
  max_workers: 1
  vectorized: false
generator:
  name: random
  supports_batch_generation: true
  supports_multi_objective: true
max_evaluations: null
serialize_inline: false
serialize_torch: false
strict: true
vocs:
  constants: {}
  constraints: {}
  objectives: {}
  observables:
  - my_pv
  variables:
    x1:
    - 0.0
    - 1.0
    x2:
    - 0.0
    - 1.0
    x3:
    - 0.0
    - 1.0


### Initialization

In [6]:
from epics import caget_many
from xopt.utils import get_local_region
# get current point
current_value = dict(zip(X.vocs.variable_names, caget_many(X.vocs.variable_names)))

# get small region around current point to sample
random_sample_region = get_local_region(current_value,X.vocs, fraction=0.1)

# random sample in a local region
X.random_evaluate(5, custom_bounds=random_sample_region)

,x1,x2,x3
0,0.0,0.0,0.000000
1,0.0,0.0,0.111111
2,0.0,0.0,0.222222
3,0.0,0.0,0.333333
4,0.0,0.0,0.444444
...,...,...,...
995,1.0,1.0,0.555556
996,1.0,1.0,0.666667
997,1.0,1.0,0.777778
998,1.0,1.0,0.888889


In [ ]:
# examine the data stored in Xopt
X.data

In [ ]:
# run optimization
for i in range(10):
    print(i)
    X.step()

In [ ]:
# visualize result
X.data.plot(y=X.vocs.objective_names[0])